In [53]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import dataset as dd # custom dataset class
import unet
from unet import UNet, shortcuts

%load_ext autoreload
%autoreload 2
%matplotlib inline

# Cleaning up variables to prevent loading data multiple times (which may cause memory issue)
try:
   del X_train, y_train
   del X_test, y_test
   print('Clear previously loaded data.')
except:
   pass

# Load the raw CIFAR-10 data.
im_ref, im_us = dd.get_dataset(1)
X_train, y_train = (im_us,  im_ref)
X_test, y_test = (X_train, y_train)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Clear previously loaded data.


In [54]:
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

('Training data shape: ', (320, 320, 256, 8))
('Training labels shape: ', (320, 320, 256, 1))
('Test data shape: ', (320, 320, 256, 8))
('Test labels shape: ', (320, 320, 256, 1))


In [55]:
# Define layers
C1 = shortcuts.conv(filters=16, kernel_size=(3, 3), padding='same', strides=(1,1), activation='relu')
M1 = shortcuts.maxpool(pool_size=(2, 2), strides=(2, 2), padding='same')
C2 = shortcuts.conv(filters=32, kernel_size=(3, 3), padding='same', strides=(1,1), activation='relu')

# "un" versions are my name for the upsampling path
# "unconv" automatically concatenates the output from the corresponding
# layer in the contracting network
C3 = shortcuts.unconv(filters=32, kernel_size=(3, 3), padding='same', strides=(1, 1), activation='relu')
U1 = shortcuts.unmaxpool(size=(2, 2))
C4 = shortcuts.unconv(filters=16, kernel_size=(3, 3), padding='same', strides=(1,1), activation='relu')
D1 = shortcuts.fc(1)

input_shape = X_train.shape[1:]
U = UNet(input_shape, C1, M1, C2, C3, U1, C4, D1)

In [56]:
U.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 320, 256, 8)       0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 320, 256, 16)      1168      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 160, 128, 16)      0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 160, 128, 32)      4640      
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 160, 128, 32)      9248      
_________________________________________________________________
up_sampling2d_10 (UpSampling (None, 320, 256, 32)      0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 320, 256, 16)      4624      
__________

In [52]:
U.train(X_train, y_train)

Epoch 1/1
320/320 [==============================] - 51s 160ms/step - loss: 0.0015 - mean_squared_error: 0.0015
